In [2]:
search_for = 132
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004259586334228516
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 1268606955
type: [1, 1, 1, 1, 132] 132
cases of this type: 303595776
100000 62.772117881951786
200000 102.19674677117575
300000 102.40865254552591
400000 100.82244664004591
500000 100.32140739367998
600000 104.38225703980085
700000 101.66149919286958
800000 98.61592890368735
900000 98.92694491585063
1000000 100.30247756267674
1100000 98.33117025761324
1200000 26.40887102283502
1300000 98.10609541700248
1400000 98.66557577690415
1500000 97.67330946585011
1600000 96.8584545577911
1700000 99.91976486001585
1800000 104.20983644269772
1900000 98.4328520858025
2000000 97.66583407222264
2100000 99.9681306280979
2200000 100.16242898783412
2300000 77.78707917534048
2400000 47.461897886895386
2500000 94.86978107741622
2600000 94.44573014316511
2700000 93.6204659017139
2800000 93.263997750726
2900000 96.20951068038457
3000000 94.888708565

29600000 103.58322880601285
29700000 69.66055864979678
29800000 76.28368817651022
29900000 106.30862508372597
30000000 100.94331003444101
30100000 97.92539246768577
30200000 42.29480814895699
30300000 101.34557270153515
30400000 102.25494697386719
30500000 100.24581131230278
30600000 97.33986225479988
30700000 99.27496948921889
30800000 96.03393089112528
30900000 93.99978491919313
31000000 88.63201454974028
31100000 26.361245326594236
31200000 96.93400511478974
31300000 92.95453226537686
31400000 95.20062739538552
31500000 99.21208356906993
31600000 99.56219266555154
31700000 98.2481534583591
31800000 100.9113212503472
31900000 99.61402897201447
32000000 56.291802669934214
32100000 87.3648357574903
32200000 103.75801719683467
32300000 105.6570160471931
32400000 107.7798504411615
32500000 33.998658162073504
32600000 107.82138667727948
32700000 106.90975475968503
32800000 102.35700879666166
32900000 104.55251802993854
33000000 103.25941652480081
33100000 99.92085832379786
33200000 97.883

59900000 101.6341686086463
60000000 100.86160917951831
60100000 104.85915280139074
60200000 102.38709360164556
60300000 37.27377586558191
60400000 98.53148176608025
60500000 95.83763409811446
60600000 95.27686352778059
60700000 91.44853557257181
60800000 90.25528014843799
60900000 90.78775455847297
61000000 23.43894829740405
61100000 94.99706605924445
61200000 91.22076756020748
61300000 90.65691076612693
61400000 90.3636710771744
61500000 97.55850902222782
61600000 98.63695909635304
61700000 37.16757730996437
61800000 106.96204190955017
61900000 104.42013845198963
62000000 100.59942402909002
62100000 105.84290841694897
62200000 97.39215702953457
62300000 96.87866692208415
62400000 98.63820206738407
62500000 96.36367446039016
62600000 50.339010148131976
62700000 81.60113876078996
62800000 91.18418272115674
62900000 89.36795732089192
63000000 83.39329920453909
63100000 86.01914813546529
63200000 87.2386981619091
63300000 23.451882325215102
63400000 92.61289310498087
63500000 86.536129166

90300000 80.2349742573961
90400000 45.862137264314576
90500000 56.58931634102243
90600000 73.09126944155824
90700000 76.33031803065876
90800000 75.73092715159797
90900000 19.72522646478505
91000000 79.97699524272066
91100000 75.75433348061136
91200000 80.39576902466851
91300000 70.23116897795244
91400000 43.12032205313656
91500000 78.30721051991169
91600000 82.68898128745583
91700000 80.63429684991853
91800000 79.45828992393957
91900000 79.66359772415295
92000000 85.57953476068269
92100000 81.49408366041418
92200000 86.65567289194652
92300000 88.45752042572244
92400000 82.58069873384235
92500000 80.73775465193226
92600000 72.56815472107581
92700000 56.64911720079114
92800000 52.91438979884424
92900000 77.70274402007009
93000000 82.23846956090686
93100000 78.43229452463322
93200000 19.88690988316662
93300000 73.33610271825616
93400000 65.95684164225213
93500000 74.32081192155105
93600000 60.00971257062079
93700000 52.579994469547344
93800000 75.62521533023762
93900000 78.68139534142014


119900000 44.034260188484204
120000000 57.9794846955072
120100000 50.88456950185093
120200000 48.115554822088534
120300000 39.18107753407205
120400000 47.618337567321426
120500000 31.719495103377874
120600000 23.81941549673745
120700000 59.20873732005577
120800000 10.51621053774473
120900000 50.989639202477576
121000000 34.68775268132563
121100000 37.59118360292003
121200000 51.808684574712075
121300000 54.58660719000098
121400000 40.69033310916239
121500000 46.309598221921
121600000 46.99713834167597
121700000 52.59021331525079
121800000 62.87042162290594
121900000 45.56656448562871
122000000 62.93515628907005
122100000 53.54418774644479
122200000 40.46109913218031
122300000 47.28735331126945
122400000 38.30755250428585
122500000 46.427928345190104
122600000 53.34500378450649
122700000 38.423439748324185
122800000 35.51059642256571
122900000 30.3629179005745
123000000 46.412624275724944
123100000 16.271065395562275
123200000 42.761614816295086
123300000 27.1006326886458
123400000 54.0

...

1189600000 0.857551524220417
1189700000 3.463850480992814
1189800000 4.859517925697525
1189900000 2.980906238262021
1190000000 2.6118512704564365
1190100000 3.302844584367603
1190200000 2.9505573924351203
1190300000 3.255356397679667
1190400000 3.981129382409851
1190500000 3.741003041341868
1190600000 2.9094481182773375
1190700000 3.6173583262572193
1190800000 3.0657375915131766
1190900000 2.7511039452795756
1191000000 3.0323835569092927
1191100000 3.466464955876582
1191200000 3.09482440769682
1191300000 3.179055572041809
1191400000 3.3859582222852307
1191500000 3.3256501184885403
1191600000 4.2803890791901145
best so far: 0
type: [1, 4, 1, 3, 11] 132
cases of this type: 1581228
1191700000 2.4834949603804715
1191800000 1.6281757950813904
1191900000 1.1436202734123766
1192000000 1.4015419986550177
1192100000 1.6593949465103814
1192200000 1.2676562450266977
1192300000 1.479321103141513
1192400000 1.8224488087046744
1192500000 1.5961542916505138
1192600000 1.4792139341005086
1192700000 1.

1203400000 3.742053096849017
1203500000 2.9665591565469107
1203600000 3.7746124290516074
1203700000 3.942005603715966
1203800000 2.975652733503325
1203900000 3.9670423723168473
1204000000 3.850755867494395
1204100000 2.6347677421589824
1204200000 4.112733091213683
1204300000 3.500048814582298
1204400000 2.877938234257585
1204500000 3.7448522053079905
1204600000 2.9799118185894056
1204700000 2.7546149294248257
1204800000 3.9289899193539912
1204900000 3.0533802805972665
1205000000 3.439555402647015
1205100000 3.8487034595768748
1205200000 2.698397634668281
1205300000 3.3707547978852688
1205400000 3.854139895026207
1205500000 2.787900592372702
1205600000 3.7090403226531907
1205700000 3.7434846062076486
1205800000 2.152507432905257
1205900000 4.123310758786848
1206000000 4.1389492995646995
1206100000 2.339477126459698
1206200000 4.128280768897746
1206300000 3.6203000145463053
1206400000 2.138804279165225
1206500000 3.8808834809533455
1206600000 3.644805113459269
1206700000 2.58934476507081

1230600000 1.1875087515761555
1230700000 1.180188261649251
1230800000 0.78785792387561
1230900000 1.088922949455748
1231000000 1.1037801920918988
1231100000 0.6766829253739483
1231200000 1.1130163584083717
1231300000 1.1150986645704077
1231400000 0.760930278892835
1231500000 1.1624609302923679
1231600000 1.173635028373109
1231700000 0.7301718975024488
1231800000 1.2118150898076891
1231900000 1.0886029817822442
1232000000 0.7470375676928792
1232100000 1.0970228598062197
1232200000 0.9859721490440071
1232300000 0.7961943255997896
1232400000 1.0064051729018986
1232500000 0.9680838380452064
1232600000 0.8527136169001891
1232700000 1.2097938802643318
1232800000 0.9522841100711193
1232900000 0.962180570581701
best so far: 0
type: [2, 1, 1, 6, 11] 132
cases of this type: 3162456
1233000000 1.142333512363116
1233100000 0.6695670056211354
1233200000 0.9365693538396722
1233300000 1.0935154344362317
1233400000 0.7725890564563473
1233500000 0.8726412756146457
1233600000 1.0318083737107018
12337000

1252200000 0.1554586483054161
1252300000 0.5019402378852831
1252400000 0.5221757256383002
1252500000 0.4607123985856374
1252600000 0.5202375734944973
1252700000 0.5305936812706655
1252800000 0.4898414123855134
1252900000 0.46370339507773184
1253000000 0.46080989734005595
1253100000 0.42219238298279044
1253200000 0.4961049206707114
best so far: 0
type: [2, 2, 1, 3, 11] 132
cases of this type: 790614
1253300000 0.3908022517786655
1253400000 0.2743466221771439
1253500000 0.2516519888389806
1253600000 0.30560908171786205
1253700000 0.262959770972987
1253800000 0.30909831813631455
1253900000 0.3233217458105882
1254000000 0.2566467468418181
best so far: 0
type: [2, 2, 1, 11, 3] 132
cases of this type: 215622
1254100000 0.2800290393928952
1254200000 0.22921644219045173
best so far: 0
type: [2, 2, 1, 33, 1] 132
cases of this type: 71874
1254300000 0.1799757429254949
best so far: 0
type: [2, 2, 3, 1, 11] 132
cases of this type: 263538
1254400000 0.25300263303660686
1254500000 0.1510483555970721

1266000000 0.041071510025779405
best so far: 0
type: [3, 2, 2, 11, 1] 132
cases of this type: 10648
best so far: 0
type: [3, 2, 11, 1, 2] 132
cases of this type: 3872
best so far: 0
type: [3, 2, 11, 2, 1] 132
cases of this type: 1936
best so far: 0
type: [3, 2, 22, 1, 1] 132
cases of this type: 968
best so far: 0
type: [3, 4, 1, 1, 11] 132
cases of this type: 58564
best so far: 0
type: [3, 4, 1, 11, 1] 132
cases of this type: 5324
best so far: 0
type: [3, 4, 11, 1, 1] 132
cases of this type: 484
best so far: 0
type: [3, 11, 1, 1, 4] 132
cases of this type: 2816
best so far: 0
type: [3, 11, 1, 2, 2] 132
cases of this type: 1408
best so far: 0
type: [3, 11, 1, 4, 1] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 1, 2] 132
cases of this type: 704
best so far: 0
type: [3, 11, 2, 2, 1] 132
cases of this type: 352
best so far: 0
type: [3, 11, 4, 1, 1] 132
cases of this type: 176
best so far: 0
type: [3, 22, 1, 1, 2] 132
cases of this type: 352
best so far: 0
type: [3, 22, 1, 2, 